In [206]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import json
import pandasql as psql
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 500
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Data Wrangling

In [207]:
# Initialize an empty list to store the DataFrames
# Read JSON data from the file line by line

data_list = []
dfs = []

with open('receipts.json/receipts.json') as f:
    for line in f:
        data_list.append(json.loads(line))

In [208]:
# Normalize and flatten the nested JSON for each object
# Concatenate all the DataFrames

for data in data_list:
    if 'rewardsReceiptItemList' in data:
        df = pd.json_normalize(data, 'rewardsReceiptItemList', 
                               [['_id','$oid'], 'bonusPointsEarned', 'bonusPointsEarnedReason', ['createDate','$date'] , 
                                ['dateScanned','$date'], 'finishedDate', 'modifyDate', 'pointsAwardedDate', 'pointsEarned',
                                'purchaseDate', 'purchasedItemCount', 'rewardsReceiptStatus', 'totalSpent', 'userId'], 
                               record_prefix='item_', errors='ignore')
        dfs.append(df)
receipts_df = pd.concat(dfs, ignore_index=True)

In [209]:
receipts_df.head()

,item_barcode,item_description,item_finalPrice,item_itemPrice,item_needsFetchReview,item_partnerItemId,item_preventTargetGapPoints,item_quantityPurchased,item_userFlaggedBarcode,item_userFlaggedNewItem,item_userFlaggedPrice,item_userFlaggedQuantity,_id.$oid,bonusPointsEarned,bonusPointsEarnedReason,createDate.$date,dateScanned.$date,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId,item_needsFetchReviewReason,item_pointsNotAwardedReason,item_pointsPayerId,item_rewardsGroup,item_rewardsProductPartnerId,item_userFlaggedDescription,item_originalMetaBriteBarcode,item_originalMetaBriteDescription,item_brandCode,item_competitorRewardsGroup,item_discountedItemPrice,item_originalReceiptItemText,item_itemNumber,item_originalMetaBriteQuantityPurchased,item_pointsEarned,item_targetPrice,item_competitiveProduct,item_originalFinalPrice,item_originalMetaBriteItemPrice,item_deleted,item_priceAfterCoupon,item_metabriteCampaignId
0,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.00,4011,True,26.00,5.00,5ff1e1eb0a720f0523000575,500,"Receipt number 2 completed, bonus point schedu...",1609687531000,1609687531000,{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5,FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.00,NaN,NaN,NaN,NaN,5ff1e1bb0a720f052300056b,150,"Receipt number 5 completed, bonus point schedu...",1609687483000,1609687483000,{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2,FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.00,028400642255,True,10.00,1.00,5ff1e1bb0a720f052300056b,150,"Receipt number 5 completed, bonus point schedu...",1609687483000,1609687483000,{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2,FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,USER_FLAGGED,Action not allowed for user and CPG,5332f5fbe4b03c9a25efd0ba,DORITOS SPICY SWEET CHILI SINGLE SERVE,5332f5fbe4b03c9a25efd0ba,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,True,26.00,3.00,5ff1e1f10a720f052300057a,5,All-receipts receipt bonus,1609687537000,1609687537000,NaN,{'$date': 1609687542000},NaN,5,{'$date': 1609632000000},1,REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.00,4011,True,28.00,4.00,5ff1e1ee0a7214ada100056f,5,All-receipts receipt bonus,1609687534000,1609687534000,{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4,FINISHED,28.00,5ff1e1eacfcf6c399c274ae6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [210]:
#select the important columns

receipts_df_final = receipts_df[['item_barcode', 'item_finalPrice', 'item_itemPrice', '_id.$oid', 'createDate.$date',  
                           'purchasedItemCount', 'totalSpent', 'userId', 'item_itemNumber', 'item_brandCode',
                                 'dateScanned.$date', 'rewardsReceiptStatus']]

In [211]:
#rename columns

receipts_df_final.rename(columns={'_id.$oid': 'id'}, inplace=True)
receipts_df_final.rename(columns={'createDate.$date': 'createTimestamp'}, inplace=True)
receipts_df_final.rename(columns={'dateScanned.$date': 'dateScannedTimestamp'}, inplace=True)

In [212]:
receipts_df_final.head()

,item_barcode,item_finalPrice,item_itemPrice,id,createTimestamp,purchasedItemCount,totalSpent,userId,item_itemNumber,item_brandCode,dateScannedTimestamp,rewardsReceiptStatus
0,4011,26.00,26.00,5ff1e1eb0a720f0523000575,1609687531000,5,26.00,5ff1e1eacfcf6c399c274ae6,NaN,NaN,1609687531000,FINISHED
1,4011,1,1,5ff1e1bb0a720f052300056b,1609687483000,2,11.00,5ff1e194b6a9d73a3a9f1052,NaN,NaN,1609687483000,FINISHED
2,028400642255,10.00,10.00,5ff1e1bb0a720f052300056b,1609687483000,2,11.00,5ff1e194b6a9d73a3a9f1052,NaN,NaN,1609687483000,FINISHED
3,NaN,NaN,NaN,5ff1e1f10a720f052300057a,1609687537000,1,10.00,5ff1e1f1cfcf6c399c274b0b,NaN,NaN,1609687537000,REJECTED
4,4011,28.00,28.00,5ff1e1ee0a7214ada100056f,1609687534000,4,28.00,5ff1e1eacfcf6c399c274ae6,NaN,NaN,1609687534000,FINISHED


In [213]:
# Initialize an empty list to store the DataFrames

dfs = []
data_list = []

with open('brands.json/brands.json') as f:
    for line in f:
        data_list.append(json.loads(line))

In [214]:
# Normalize the JSON data
# Concatenate all the DataFrames

for data in data_list:
    df = pd.json_normalize(data)
    dfs.append(df)
    
brands_df = pd.concat(dfs, ignore_index=True)

In [215]:
#Rename columns

brands_df.rename(columns={'_id.$oid': 'id'}, inplace=True)
brands_df.rename(columns={'cpg.$id.$oid': 'cpgId'}, inplace=True)
brands_df.rename(columns={'cpg.$ref': 'cpgRef'}, inplace=True)

In [216]:
brands_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   barcode       1167 non-null   object
 1   category      1012 non-null   object
 2   categoryCode  517 non-null    object
 3   name          1167 non-null   object
 4   topBrand      555 non-null    object
 5   id            1167 non-null   object
 6   cpgId         1167 non-null   object
 7   cpgRef        1167 non-null   object
 8   brandCode     933 non-null    object
dtypes: object(9)
memory usage: 82.2+ KB


In [217]:
brands_df.head()

,barcode,category,categoryCode,name,topBrand,id,cpgId,cpgRef,brandCode
0,511111019862,Baking,BAKING,test brand @1612366101024,False,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs,NaN
1,511111519928,Beverages,BEVERAGES,Starbucks,False,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS
2,511111819905,Baking,BAKING,test brand @1612366146176,False,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146176
3,511111519874,Baking,BAKING,test brand @1612366146051,False,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146051
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1612366146827


In [218]:
# Initialize an empty list to store the data
# Read JSON data from the file line by line

data_list = []
dfs = []

with open('users.json/users.json') as f:
    for line in f:
        data_list.append(json.loads(line))

In [219]:
# Normalize the JSON data
# Concatenate all the DataFrames

for data in data_list:
    df = pd.json_normalize(data)
    dfs.append(df)
    
users_df = pd.concat(dfs, ignore_index=True)

In [220]:
# Rename the columns

users_df.rename(columns={'_id.$oid': 'id'}, inplace=True)
users_df.rename(columns={'createdDate.$date': 'createdTimestamp'}, inplace=True)
users_df.rename(columns={'lastLogin.$date': 'lastLoginTimestamp'}, inplace=True)

In [221]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 495 entries, 0 to 494
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   active              495 non-null    bool   
 1   role                495 non-null    object 
 2   signUpSource        447 non-null    object 
 3   state               439 non-null    object 
 4   id                  495 non-null    object 
 5   createdTimestamp    495 non-null    int64  
 6   lastLoginTimestamp  433 non-null    float64
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 23.8+ KB


In [222]:
#A function to convert the timestamp into datetime

def convert_timestamp(df, convert_col_name, new_col_name):
    df[new_col_name] = pd.to_datetime(df[convert_col_name], unit='ms')
    return df

In [223]:
#Converting the the timestamps for receipts

receipts_df_final = convert_timestamp(receipts_df_final, 'createTimestamp', 'createDate')
receipts_df_final = convert_timestamp(receipts_df_final, 'dateScannedTimestamp', 'dateScannedDate')
receipts_df_final.head()

,item_barcode,item_finalPrice,item_itemPrice,id,createTimestamp,purchasedItemCount,totalSpent,userId,item_itemNumber,item_brandCode,dateScannedTimestamp,rewardsReceiptStatus,createDate,dateScannedDate
0,4011,26.00,26.00,5ff1e1eb0a720f0523000575,1609687531000,5,26.00,5ff1e1eacfcf6c399c274ae6,NaN,NaN,1609687531000,FINISHED,2021-01-03 15:25:31,2021-01-03 15:25:31
1,4011,1,1,5ff1e1bb0a720f052300056b,1609687483000,2,11.00,5ff1e194b6a9d73a3a9f1052,NaN,NaN,1609687483000,FINISHED,2021-01-03 15:24:43,2021-01-03 15:24:43
2,028400642255,10.00,10.00,5ff1e1bb0a720f052300056b,1609687483000,2,11.00,5ff1e194b6a9d73a3a9f1052,NaN,NaN,1609687483000,FINISHED,2021-01-03 15:24:43,2021-01-03 15:24:43
3,NaN,NaN,NaN,5ff1e1f10a720f052300057a,1609687537000,1,10.00,5ff1e1f1cfcf6c399c274b0b,NaN,NaN,1609687537000,REJECTED,2021-01-03 15:25:37,2021-01-03 15:25:37
4,4011,28.00,28.00,5ff1e1ee0a7214ada100056f,1609687534000,4,28.00,5ff1e1eacfcf6c399c274ae6,NaN,NaN,1609687534000,FINISHED,2021-01-03 15:25:34,2021-01-03 15:25:34


In [224]:
#Converting the the timestamps for users

users_df = convert_timestamp(users_df, 'createdTimestamp', 'createdDate')
users_df = convert_timestamp(users_df, 'lastLoginTimestamp', 'lastLoginDate')
users_df.head()

,active,role,signUpSource,state,id,createdTimestamp,lastLoginTimestamp,createdDate,lastLoginDate
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,"1,609,687,537,858.00",2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872
1,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,"1,609,687,537,858.00",2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872
2,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,"1,609,687,537,858.00",2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872
3,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,1609687530554,"1,609,687,530,597.00",2021-01-03 15:25:30.554,2021-01-03 15:25:30.596999936
4,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,1609687444800,"1,609,687,537,858.00",2021-01-03 15:24:04.800,2021-01-03 15:25:37.857999872


### Answers
#### Q1 What are the top 5 brands by receipts scanned for most recent month?
#### Q2 How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?

In [225]:
query = """
WITH prev_mon_cte AS
(
         SELECT   item_brandcode,
                  Count(DISTINCT(id)) AS receipt_count_prev_mon
         FROM     receipts_df_final
         WHERE    datescanneddate BETWEEN '2021-01-01' AND '2021-01-31'
         AND      item_brandcode != 'None'
         GROUP BY item_brandcode
         ORDER BY receipt_count_prev_mon DESC)
SELECT    rf.item_brandcode,
          Count(DISTINCT(rf.id)) AS receipt_count_latest_mon,
          pm.receipt_count_prev_mon
FROM      receipts_df_final rf
LEFT JOIN prev_mon_cte pm
ON        pm.item_brandcode = rf.item_brandcode
WHERE     rf.datescanneddate BETWEEN '2021-02-01' AND '2021-02-28'
AND       rf.item_brandcode != 'None'
GROUP BY  rf.item_brandcode
ORDER BY  receipt_count_latest_mon DESC limit 5;
"""

result = psql.sqldf(query, locals())
result

,item_brandCode,receipt_count_latest_mon,receipt_count_prev_mon
0,BRAND,3,19.00
1,MISSION,2,16.00
2,VIVA,1,NaN


* In the above table we can see the three brands with their latest month(February 2021) receipt count and their previous month(January 2021) receipt count. 
* There are only 3 when 5 were asked, as the latest month has less data.

#### Q3 When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [226]:
query = """
SELECT Avg(CASE
             WHEN rewardsreceiptstatus = 'FINISHED' THEN totalspent
             ELSE NULL
           END) AS Finished_Amt,
       Avg(CASE
             WHEN rewardsreceiptstatus = 'REJECTED' THEN totalspent
             ELSE NULL
           END) AS Rejected_Amt
FROM   receipts_df_final;
"""

result = psql.sqldf(query, locals())
result

,Finished_Amt,Rejected_Amt
0,"1,244.79",19.90


* Assuming the question ment 'Finished' and 'Rejected', above are the average amounts for both of them.
* Here Finished amount is far greater than Rejected amount based on total spent.

#### Q4 When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?


In [227]:
query = """
SELECT Sum(CASE
             WHEN rewardsreceiptstatus = 'FINISHED' THEN purchaseditemcount
             ELSE 0
           END) AS Finished_item_count,
       Sum(CASE
             WHEN rewardsreceiptstatus = 'REJECTED' THEN purchaseditemcount
             ELSE 0
           END) AS Rejected_item_count
FROM   receipts_df_final; 
"""

result = psql.sqldf(query, locals())
result

,Finished_item_count,Rejected_item_count
0,1364998,740


* Again, assuming the question ment 'Finished' and 'Rejected', above are the item counts for both of them.
* Here Finished item count is far greater than Rejected count based on purchased Item Count.

#### Q5 Which brand has the most spend among users who were created within the past 6 months?

In [228]:
query = """
SELECT rf.item_brandcode,
       Sum(rf.totalspent) total_Brand_Spent
FROM   receipts_df_final rf
WHERE  rf.userid IN (SELECT id
                     FROM   users_df
                     WHERE  createddate BETWEEN '2020-09-01' AND '2021-02-31'
                            AND role = 'consumer')
       AND rf.item_brandcode != 'None'
GROUP  BY 1
ORDER  BY 2 DESC
LIMIT  1; 
"""

result = psql.sqldf(query, locals())
result

,item_brandCode,total_Brand_Spent
0,BEN AND JERRYS,"197,337.68"


* 'BEN AND JERRYS' has the most spend among users who were created within the last 6 months from the max date

#### Figuring out the Primark Key for Receipts and users dataset after flattening(unnesting)

In [229]:
# Tried all selected columns as composit primary key for receipt dataset(after flattening).
# But it is not working as there are duplicate rows. We might need to create a new column as a primary key

query = """
SELECT id,
       item_barcode,
       item_finalprice,
       item_itemprice,
       createtimestamp,
       purchaseditemcount,
       totalspent,
       userid,
       item_itemnumber,
       item_brandcode,
       datescannedtimestamp,
       createdate,
       Count(*) as duplicate_count
FROM   receipts_df_final
GROUP  BY id,
          item_barcode,
          item_finalprice,
          item_itemprice,
          createtimestamp,
          purchaseditemcount,
          totalspent,
          userid,
          item_itemnumber,
          item_brandcode,
          datescannedtimestamp,
          createdate
HAVING Count(*) > 1; 
"""

result = psql.sqldf(query, locals())
result

,id,item_barcode,item_finalPrice,item_itemPrice,createTimestamp,purchasedItemCount,totalSpent,userId,item_itemNumber,item_brandCode,dateScannedTimestamp,createDate,duplicate_count
0,5f9c74f70a7214ad07000037,None,None,None,1604089079000,11,1.00,5f9c74f7c88c1415cbddb839,None,None,1604089079000,2020-10-30 20:17:59.000000,10
1,5f9c74f90a7214ad07000038,034100573065,1.00,1.00,1604089081000,6,14.00,5f9c74e3f1937815bd2c1d73,None,None,1604089081000,2020-10-30 20:18:01.000000,3
2,5fa5ad370a720f05ef000089,034100573065,29.00,29.00,1604693303000,11,291.00,5fa5ad376a26f611e71ab5ef,None,None,1604693303000,2020-11-06 20:08:23.000000,10
3,5fa5b0ca0a720f05ef0000bf,034100573065,1.00,1.00,1604607818000,6,14.00,5fa5b0b720dc5111dd86dcc1,None,None,1604607818000,2020-11-05 20:23:38.000000,3
4,5fa8d5730a7214adc30001c3,None,None,None,1604900211000,11,1.00,5fa8d573770b516dda4f4d21,None,None,1604900211000,2020-11-09 05:36:51.000000,10
5,5fa8d5790a7214adc30001c4,034100573065,1.00,1.00,1604900217000,6,14.00,5fa8d542b055fd1dfdf09517,None,None,1604900217000,2020-11-09 05:36:57.000000,3
6,5fb1bd510a7214ad890000e3,None,None,None,1605483857000,11,1.00,5fb1bd519c09d811e5211cef,None,None,1605483857000,2020-11-15 23:44:17.000000,10
7,5fb2da580a7214ad89000114,None,None,None,1605556824000,11,1.00,5fb2da582357ec11d4ecc403,None,None,1605556824000,2020-11-16 20:00:24.000000,10
8,5ff1e1b40a7214ada100055b,None,None,None,1609687476000,1,1.00,5ff1e1b4cfcf6c399c274a54,None,None,1609687476000,2021-01-03 15:24:36.000000,10
9,5ff1e1b60a7214ada100055c,034100573065,29,29,1609687478000,10,290.00,5ff1e194b6a9d73a3a9f1052,None,None,1609687478000,2021-01-03 15:24:38.000000,10


In [230]:
# id supposed to be the primary key of this data but there are duplicates for it.
# Tried with various composit primary keys as well, dosen't seem to work. We need to throughly look into data.

query = """
SELECT id,
       createdTimestamp,
       Count(*) as duplicate_count
FROM   users_df
GROUP  BY id,
          createdTimestamp
HAVING Count(*) > 1; 
"""

result = psql.sqldf(query, locals())
result

,id,createdTimestamp,duplicate_count
0,54943462e4b07e684157a532,1418998882381,20
1,59c124bae4b0299e55b0f330,1505830074302,18
2,5a43c08fe4b014fd6b6a0612,1514389647059,8
3,5fa41775898c7a11a6bcef3e,1604589429396,18
4,5fb0a078be5fc9775c1f3945,1605410936818,2
5,5fbc35711d967d1222cbfefc,1606169969509,3
6,5fc961c3b8cfca11a077dd33,1607033283936,20
7,5ff1e194b6a9d73a3a9f1052,1609687444800,11
8,5ff1e1eacfcf6c399c274ae6,1609687530554,4
9,5ff36a3862fde912123a4460,1609787960352,5
